In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

2020-12-21 17:28:48,253 - NumExpr defaulting to 8 threads.


In [2]:
path = '../raw_data/'

In [3]:
at_raw = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv('../raw_data/SCS alerts Nov.csv')

In [4]:
# HOTFIX - remove odd day
at_raw = at_raw[~at_raw['DAY'].isin([9])]

In [5]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2459057445616) was called 1 times


In [6]:
at = feat.pre_process_AT(at_raw)

Running: Function "pre_process_AT" (id=2459057444464) was called 1 times


In [7]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2459057445328) was called 1 times


In [8]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2459057446480) was called 1 times
Running: Function "load_module_lookup" (id=2459057443744) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [9]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [89]:
mod = [1]
limit = 250

In [90]:
fa_sel_1 = feat.get_data_faults(fa_floor, modules = mod)

most_common = list(((fa_sel_1['Asset Code'].value_counts() > limit)[fa_sel_1['Asset Code'].value_counts() > limit]).index)
fa_sel = fa_sel_1[fa_sel_1['Asset Code'].isin(most_common)]

In [91]:
#fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Blue']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel, fault_agg_level= 'Asset Code', agg_type = 'count')

In [92]:
av_sel, at_sel = feat.av_at_select(av, at, remove_high_AT = True, availability_select_options = {"Module" : mod})

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

Running: Function "aggregate_availability" (id=2459057444176) was called 4 times


In [93]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, agg_level = 'None')

In [94]:
# df = (pd.merge(at_agg.reset_index(), av_agg.reset_index(), on=['timestamp', 'Module'], how='left')
#  .merge(fa_agg, on=['timestamp'], how='left')).drop(['Module', 'Blue Tote Loss', 'Grey Tote Loss'], axis=1)
# df = df.fillna(0)

In [95]:
X,y = md.gen_feat_var(df)
X = X.drop('Module', axis=1)
# Flip to availability
y=1-y

In [96]:
X_train, X_test, y_train, y_test = md.split(X,y)

#RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,num_trees=100, criterion = 'mse', max_depth=None, dtree=False)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

#XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test)

maxf = None

# X_sel_rf = md.select_features(X, y, RF_mdl, thres = -np.inf, max_feat = maxf)

#X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options={'thres': 'mean', 'max_feat': None})

#X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)


cv_R2 = md.cross_validate_r2(Linear_mdl, X, y, n_folds = 5, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(RF_mdl, X_sel_rf, y, n_folds = 10, shuffle = True, random_state = 101)

#cv_R2 = md.cross_validate_r2(XGB_mdl, X, y, n_folds = 10, shuffle = True, random_state = 101)

#md.stats_model(Linear_mdl,X_test,y_test)


Cross Validation Scores LinearRegression(): 
 
        R2 Scores
1       0.511324
2       0.118633
3       0.403904
4       0.309821
5       0.591844
Mean    0.387105
STD     0.164730


In [97]:
model = sm.OLS(y,X)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.939
Model:                            OLS   Adj. R-squared (uncentered):              0.932
Method:                 Least Squares   F-statistic:                              134.5
Date:                Mon, 21 Dec 2020   Prob (F-statistic):                    2.08e-71
Time:                        17:41:40   Log-Likelihood:                          83.819
No. Observations:                 145   AIC:                                     -137.6
Df Residuals:                     130   BIC:                                     -92.99
Df Model:                          15                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [98]:
keep_features = results.pvalues[results.pvalues < 0.10].index

In [99]:
model = sm.OLS(y,X[keep_features])
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.938
Model:                            OLS   Adj. R-squared (uncentered):              0.934
Method:                 Least Squares   F-statistic:                              257.8
Date:                Mon, 21 Dec 2020   Prob (F-statistic):                    1.27e-78
Time:                        17:41:48   Log-Likelihood:                          81.725
No. Observations:                 145   AIC:                                     -147.5
Df Residuals:                     137   BIC:                                     -123.6
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [100]:
negs = results.params[results.params < 0].index

In [101]:
df['timestamp'].min()

Timestamp('2020-11-17 09:00:00')

In [103]:
fa[fa['Asset Code'].isin([negs]) & (fa['timestamp'] >= df['timestamp'].min())]#['Fault ID'].value_counts()

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE
145767,130231,C16: C1603BCL014 014B52 sensor fault,2020-11-20 20:39:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
124159,130234,C16: C1603BCL014 014B55 sensor fault,2020-11-19 08:01:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
136517,130234,C16: C1603BCL014 014B55 sensor fault,2020-11-22 00:45:00,C16,P03,1.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
137649,130233,C16: C1603BCL014 014B54 sensor fault,2020-11-21 20:52:00,C16,P03,2.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
131696,130233,C16: C1603BCL014 014B54 sensor fault,2020-11-22 15:33:00,C16,P03,2.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170158,130233,C16: C1603BCL014 014B54 sensor fault,2020-11-25 13:46:00,C16,P03,209.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
122802,130233,C16: C1603BCL014 014B54 sensor fault,2020-11-19 02:14:00,C16,P03,229.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
154251,130233,C16: C1603BCL014 014B54 sensor fault,2020-11-23 16:37:00,C16,P03,247.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
170156,130233,C16: C1603BCL014 014B54 sensor fault,2020-11-25 10:24:00,C16,P03,298.0,3428,C1603BCL014,Blue,Outside,C1603,False,16,0,NaN
